In [31]:
import random
from collections import defaultdict

In [ ]:
def conflict_free_courses(schedule, student_courses):
    """ Evaluation function
    """
    #count how many preferred courses each student gets without time conflicts
    total_conflict_free_courses = 0

    for _, preferred_courses in student_courses.items():
        times = [schedule[course] for course in preferred_courses if course in schedule]
        unique_times = set(times)

        # count only non-conflicting courses
        conflict_free_courses = len(unique_times)
        total_conflict_free_courses += conflict_free_courses

    return total_conflict_free_courses

In [ ]:
# constraints
def is_valid_solution(schedule):
    """ Make sure that the schedule produced follows all constraints:
    
        - no time slot has more than 3 courses
        - SCOPE is on Wednesdays
        - ...
    """
    
    time_slot_counts = defaultdict(int)

    for course, time_slot in schedule.items():
        # check first year classes not on wed
        if 'FRST' in course and time_slot == 5:
            return False
        time_slot_counts[time_slot] += 1

    # check no more than 3 classes per slot
    if not all(count <= 3 for count in time_slot_counts.values()):
        return False
    

    # scope on wed
    if 'SCOPE' in schedule.keys():
        if schedule['SCOPE'] != 5:
            return False


    return True

In [ ]:
def generate_solutions(schedule, time_slots):
    """Generate neighboring solutions by moving or swapping course time slots."""
    neighbors = []
    courses = list(schedule.keys())

    for _ in range(len(courses)):  # Generate multiple candidates
        new_schedule = schedule.copy()

        if random.random() < 0.5:  # Move a course to a different time slot
            course = random.choice(courses)
            new_schedule[course] = random.choice(time_slots)

        else:  # Swap two courses' time slots
            c1, c2 = random.sample(courses, 2)
            new_schedule[c1], new_schedule[c2] = new_schedule[c2], new_schedule[c1]

        # Only add valid schedules
        if is_valid_solution(new_schedule):
            neighbors.append(new_schedule)

    return neighbors

In [ ]:
def tabu_search(courses, student_courses, time_slots, max_iterations=100, tabu_size=10):
    """Optimize course scheduling using Tabu Search."""
    
    # Initialize random valid schedule
    while True:
        current_schedule = {course: random.choice(time_slots) for course in courses}
        if is_valid_solution(current_schedule):
            break
    print(f"the current schedule is {current_schedule}")

    best_schedule = current_schedule
    best_score = conflict_free_courses(current_schedule, student_courses)

    tabu_list = []

    for _ in range(max_iterations):
        neighbors = generate_solutions(current_schedule, time_slots)
        # print(f"the neighbors are {neighbors}")

        # Evaluate neighbors and select the best one not in the tabu list
        best_neighbor = None
        best_neighbor_score = -1

        for neighbor in neighbors:
            if neighbor not in tabu_list:
                score = conflict_free_courses(neighbor, student_courses)
                if score > best_neighbor_score:
                    best_neighbor = neighbor
                    best_neighbor_score = score

        if best_neighbor:
            current_schedule = best_neighbor
            tabu_list.append(best_neighbor)
            if len(tabu_list) > tabu_size:  # Maintain tabu list size
                tabu_list.pop(0)

            if best_neighbor_score > best_score:
                best_schedule = best_neighbor
                best_score = best_neighbor_score

    return best_schedule, best_score


In [41]:
# example
courses = ["CS101", "CS102", "MATH201", "PHYS101", "HIST202", "BIO301", "CHEM303", "SCOPE", "FRST101"]
time_slots = [1, 2, 3, 4, 5]

"""
1 - MR 1 pm 
2 - MR 2:50 pm
3 - TF 1 pm
4 - TF 2:50 pm
5 - W
"""

student_courses = {
    "Alice": {"CS101", "MATH201", "BIO301"},
    "Bob": {"CS101", "CS102", "HIST202"},
    "Charlie": {"CS102", "MATH201", "CHEM303"},
    "David": {"PHYS101", "MATH201", "BIO301"},
    "Eve": {"CHEM303", "PHYS101", "CS101"},
    "Franklin" : {"FRST101"}
}

best_schedule, best_score = tabu_search(courses, student_courses, time_slots)

print("Optimized Schedule:", best_schedule)
print("Total Conflict-Free Course Assignments:", best_score)


the current schedule is {'CS101': 2, 'CS102': 3, 'MATH201': 2, 'PHYS101': 5, 'HIST202': 2, 'BIO301': 5, 'CHEM303': 1, 'SCOPE': 5, 'FRST101': 1}
Optimized Schedule: {'CS101': 4, 'CS102': 3, 'MATH201': 2, 'PHYS101': 5, 'HIST202': 2, 'BIO301': 3, 'CHEM303': 1, 'SCOPE': 5, 'FRST101': 1}
Total Conflict-Free Course Assignments: 16


schedule example: {'CS101': 1, 'CS102': 3, 'MATH201': 2, 'PHYS101': 2, 'HIST202': 4, 'BIO301': 1, 'CHEM303': 1}

In [ ]:
# example
courses = ["CS101", "CS102", "MATH201", "PHYS101", "HIST202", "BIO301", "CHEM303"]
time_slots = [1, 2, 3, 4]

student_courses = {
    "Lily": {"CS101", "MATH201", "BIO301"},
    "Madie": {"CS101", "CS102", "HIST202"},
    "Olga": {"CS102", "MATH201", "CHEM303"},
    "Amit": {"PHYS101", "MATH201", "BIO301"},
    "Anmol": {"CHEM303", "PHYS101", "CS101"}
}

best_schedule, best_score = tabu_search(courses, student_courses, time_slots)

print("Optimized Schedule:", best_schedule)
print("Total Conflict-Free Course Assignments:", best_score)

the current schedule is {'CS101': 1, 'CS102': 3, 'MATH201': 4, 'PHYS101': 1, 'HIST202': 4, 'BIO301': 1, 'CHEM303': 4}
the neighbors are [{'CS101': 1, 'CS102': 3, 'MATH201': 4, 'PHYS101': 1, 'HIST202': 4, 'BIO301': 1, 'CHEM303': 3}, {'CS101': 4, 'CS102': 3, 'MATH201': 4, 'PHYS101': 1, 'HIST202': 4, 'BIO301': 1, 'CHEM303': 1}, {'CS101': 1, 'CS102': 3, 'MATH201': 4, 'PHYS101': 1, 'HIST202': 4, 'BIO301': 3, 'CHEM303': 4}, {'CS101': 1, 'CS102': 3, 'MATH201': 4, 'PHYS101': 1, 'HIST202': 4, 'BIO301': 1, 'CHEM303': 4}, {'CS101': 1, 'CS102': 3, 'MATH201': 4, 'PHYS101': 1, 'HIST202': 4, 'BIO301': 1, 'CHEM303': 4}, {'CS101': 4, 'CS102': 3, 'MATH201': 4, 'PHYS101': 1, 'HIST202': 4, 'BIO301': 1, 'CHEM303': 1}]
the neighbors are [{'CS101': 4, 'CS102': 3, 'MATH201': 4, 'PHYS101': 1, 'HIST202': 1, 'BIO301': 3, 'CHEM303': 4}, {'CS101': 1, 'CS102': 4, 'MATH201': 4, 'PHYS101': 1, 'HIST202': 4, 'BIO301': 3, 'CHEM303': 3}, {'CS101': 1, 'CS102': 3, 'MATH201': 4, 'PHYS101': 1, 'HIST202': 4, 'BIO301': 3, 'CHE